# Importing Libraries

In [4]:
import pandas as pd
import nltk
import re
from torchvision.datasets import ImageFolder
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from sklearn.model_selection import train_test_split
from torch import tensor
import torchvision
import torchvision.transforms.functional as TF
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import torch.optim as optim
import numpy as np
import ast
from torch.utils.data import TensorDataset, DataLoader
import os

In [5]:
df=pd.read_csv("sentiment/sentiment.csv")
df

Unnamed: 0  imgid  split                       filename  successful  \
0               0  31369  train  COCO_val2014_000000389081.jpg           1   
1               1  31369  train  COCO_val2014_000000389081.jpg           1   
2               2  31369  train  COCO_val2014_000000389081.jpg           1   
3               3  31369  train  COCO_val2014_000000389081.jpg           1   
4               4  31369  train  COCO_val2014_000000389081.jpg           1   
...           ...    ...    ...                            ...         ...   
39194       39194  24628   test  COCO_val2014_000000190705.jpg           1   
39195       39195  24628   test  COCO_val2014_000000190705.jpg           1   
39196       39196  24628   test  COCO_val2014_000000190705.jpg           1   
39197       39197  24628   test  COCO_val2014_000000190705.jpg           1   
39198       39198  24628   test  COCO_val2014_000000190705.jpg           1   

                                                  tokens  \
0      ['a', 'plate', 'of', 'delicious', 'food', 'inc...   
1      ['french', 'fries', 'are', 'not', 'a', 'health...   
2      ['the', 'plate', 'has', 'one', 'of', 'my', 'fa...   
3      ['it', 'was', 'disgusting', 'food', 'not', 'ju...   
4      ['a', 'plate', 'of', 'disgusting', 'food', 'fo...   
...                                                  ...   
39194  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39195  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39196  ['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...   
39197  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39198  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   

                                          word_sentiment  sentiment  \
0                   [0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]          1   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...          1   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...          1   
3                       [0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]          0   
4              [0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]          0   
...                                                  ...        ...   
39194   [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]          0   
39195            [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]          0   
39196  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          0   
39197   [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]          0   
39198            [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]          0   

                                                     raw  
0      a plate of delicious food including French fries.  
1      French fries are not a healthy food but it is ...  
2      The plate has one of my favorite foods on it, ...  
3             It was disgusting food, not just bad food.  
4           A plate of disgusting food found at a diner.  
...                                                  ...  
39194  A dirty bathroom that has a dirty window made ...  
39195          A dirty bathroom that has a window in it.  
39196      a towel that is on a rack in a dirty bathroom  
39197  A dirty bathroom that has a dirty window made ...  
39198          A dirty bathroom that has a window in it.  

[39199 rows x 9 columns]

In [7]:
os.chdir("sentiment")

In [8]:
os.chdir("sentiment_images")

In [11]:
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from PIL import Image

# Get list of files in the current directory
file_list = [f for f in os.listdir(os.getcwd()) if f]

# Image transformations
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load pre-trained ResNet model
resnet = torchvision.models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

# Lists to store image features and labels
image_features = []
labels = []

# Process each file
for filename in file_list:
    image = Image.open(filename)
    if image.mode == 'L':
        image = TF.to_tensor(image).expand(3, -1, -1)
    elif image.mode == 'RGB':
        image = TF.to_tensor(image)
    else:
        raise ValueError("Unsupported color mode: %s" % image.mode)
    image = image.unsqueeze(0)
    features = resnet(image)
    features = features.squeeze().detach().numpy()
    image_features.append(features)
    labels.append(filename)

# Convert lists to numpy arrays
image_features = np.array(image_features)
labels = np.array(labels)

# Print shapes of image features and labels
print("Image Features Shape:", image_features.shape)
print("Labels Shape:", labels.shape)

# Change the directory
os.chdir("..")

# Lists to store image features and filenames
filtered_image_features = []
not_present_filenames = []

# Process each filename in the dataframe
for filename in df["filename"]:
    found = False
    for i in range(len(image_features)):
        if labels[i] == filename:
            filtered_image_features.append(image_features[i])
            found = True
            break
    if not found:
        if filename not in not_present_filenames:
            not_present_filenames.append(filename)

# Remove entries with filenames not present in the image features
df = df[~df['filename'].isin(not_present_filenames)]

# Add image features to the dataframe
df["image_features"] = filtered_image_features

# Convert image features to a PyTorch tensor
train_caption_image = torch.tensor(df["image_features"].tolist())

# Print the updated shapes
print("Updated Image Features Shape:", train_caption_image.shape)

Image Features Shape: (2221, 512)
Labels Shape: (2221,)
Updated Image Features Shape: torch.Size([39109, 512])


C:\Users\USER\AppData\Local\Temp/ipykernel_1936/130307666.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["image_features"] = filtered_image_features
C:\Users\USER\AppData\Local\Temp/ipykernel_1936/130307666.py:77: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  train_caption_image = torch.tensor(df["image_features"].tolist())


In [12]:
labels=torch.tensor(df["sentiment"].tolist())
labels

tensor([1, 1, 1,  ..., 0, 0, 0])

In [13]:
import torch
from torch.nn.utils.rnn import pad_sequence

padded_sentiment = pad_sequence([torch.tensor(x) for x in [list(map(float, x[1:-1].split(", "))) for x in df["word_sentiment"].tolist()]], batch_first=True)
padded_sentiment

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.]])

# Designing Model 

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultimodalModel(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, hidden_size, num_layers):
        super(MultimodalModel, self).__init__()

        self.text_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.text_lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)

        self.visual_fc1 = nn.Linear(512, 256)
        self.visual_fc2 = nn.Linear(256, hidden_size)

        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(hidden_size + hidden_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, num_classes)

    def forward(self, text, image):
        text = self.text_embedding(text.long())
        _, (text_output, _) = self.text_lstm(text)
        text_output = text_output[-1]

        visual_output = F.relu(self.visual_fc1(image))
        visual_output = F.relu(self.visual_fc2(visual_output))

        combined = torch.cat((text_output, visual_output), dim=1)
        combined = self.dropout(combined)

        output = F.relu(self.fc1(combined))
        output = F.relu(self.fc2(output))
        output = F.relu(self.fc3(output))
        output = F.relu(self.fc4(output))
        output = F.relu(self.fc5(output))
        output = self.fc6(output)
        return output


# Training Model

In [24]:
model = MultimodalModel(num_classes=2, vocab_size=29572, embedding_dim=256, hidden_size=128, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
num_epochs = 15

dataset = TensorDataset(padded_sentiment, train_caption_image, labels)
batch_size = 300
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        padded_sentiment_batch, batch_caption_image, labels_batch = batch

        optimizer.zero_grad()
        outputs = model(padded_sentiment_batch, batch_caption_image)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / (len(dataset) // batch_size)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))

torch.save(model.state_dict(), "model2.pt")


Epoch [1/15], Loss: 0.7054
Epoch [2/15], Loss: 0.6949
Epoch [3/15], Loss: 0.6949
Epoch [4/15], Loss: 0.6948
Epoch [5/15], Loss: 0.6950
Epoch [6/15], Loss: 0.6949
Epoch [7/15], Loss: 0.6948
Epoch [8/15], Loss: 0.6950
Epoch [9/15], Loss: 0.6950
Epoch [10/15], Loss: 0.6949
Epoch [11/15], Loss: 0.6948
Epoch [12/15], Loss: 0.6949
Epoch [13/15], Loss: 0.6948
Epoch [14/15], Loss: 0.6948
Epoch [15/15], Loss: 0.6948


# Evaluation

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = MultimodalModel(num_classes=2, vocab_size=29572, embedding_dim=256, hidden_size=128, num_layers=2)
model.load_state_dict(torch.load("model2.pt"))

dataset = TensorDataset(padded_sentiment, train_caption_image, labels)
batch_size = 500
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

model.eval()
predicted_labels = []
true_labels = []

with torch.no_grad():
    for batch in dataloader:
        padded_sentiment_batch, batch_caption_image, labels_batch = batch

        outputs = model(padded_sentiment_batch, batch_caption_image)
        _, predicted = torch.max(outputs.data, 1)

        predicted_labels.extend(predicted.tolist())
        true_labels.extend(labels_batch.tolist())

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print('Accuracy on test set: {:.4f}%'.format(accuracy * 100))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 Score: {:.4f}'.format(f1))


Accuracy on test set: 54.2816%
Precision: 0.5428
Recall: 1.0000
F1 Score: 0.7037
